In [2]:
import sqlite3

In [3]:
db_path = "laminateBase.db"
conn = sqlite3.connect(db_path)

In [4]:
conn.close()

In [26]:
# raise RuntimeError("Attempted to run all cells!")

In [27]:
### DELETE WHOLE DATABASE 


In [23]:
def createTableStr(name:str, table:list) -> None:
    tableString = f"CREATE TABLE IF NOT EXISTS {name} ("
    for row in table:
        tableString += row + ", "
    
    tableString = tableString[:-2]
    tableString += ")"
    return tableString

Defining basic attribute tables

In [29]:
tb_materials = ["NAME TEXT"]

tb_usecases = ["NAME TEXT"]

tb_stackings = ["NAME TEXT"]

tb_weaves = ["NAME TEXT"]

tb_states = ["NAME TEXT"]


list_materials = ["Aramid", "Carbon", "Aluminium", "Glass"]
list_usecases = ["High stiffness", "Stiffness", "Geometry", "Strength", "High strength", "Electric isolation", "Fire safety"] 
list_stackings = ["Symmetric", "Total", "Inverted"]
list_weaves = ["Twill", "Weave"]
list_states = ["New", "Ok", "Old", "Discarded"]

Create the different basic attribute tables 

In [30]:
cur = conn.cursor()

cur.execute(createTableStr("materials", tb_materials))
cur.execute(createTableStr("usecases", tb_usecases))
cur.execute(createTableStr("stackings", tb_stackings))
cur.execute(createTableStr("weaves", tb_weaves))
cur.execute(createTableStr("states", tb_states))

cur.close()

Populate basic attribute tables

In [31]:
cur = conn.cursor()

for material in list_materials:
    cur.execute("INSERT INTO materials(name) VALUES (?)", (material,))

for usecase in list_usecases:
    cur.execute("INSERT INTO usecases(name) VALUES (?)", (usecase,))

for stacking in list_stackings:
    cur.execute("INSERT INTO stackings(name) VALUES (?)", (stacking,))

for weave in list_weaves:
    cur.execute("INSERT INTO weaves(name) VALUES (?)", (weave,))

for state in list_states:
    cur.execute("INSERT INTO states(name) VALUES (?)", (state,))
cur.close()
conn.commit()


Create Fibers table

In [35]:
tb_fibers = ["name TEXT",
             "material INT",
             "modulus REAL",
             "uts REAL",
             "rho REAL",
             "comment TEXT",
             "FOREIGN KEY(material) REFERENCES materials(rowid)"]

cur = conn.cursor()

cur.execute(createTableStr("fibers", tb_fibers))
cur.close()
conn.commit()

In [4]:
from db_sqlite3_v3 import DbHandler

test_data = {"name":"testfiber 3", "material":1, "modulus":230, "uts":45, "rho":955, "comment":"Testfiber"}
db = DbHandler("laminateBase.db")
success, msg = db.addFiber(test_data)

print(success, msg)
# cur = conn.cursor()
# cur.execute

Database connected successfully!
True Fiber added to database


Populate fibers table with dummy data

In [36]:
# list_fibers = [["Aramid test", 1, 260, 50, 6.2, "Testfiber 1"],
#                ["IMS60", 2, 300, 420, 4.2, "Testfiber 2"]]

# cur = conn.cursor()

# for fiber in list_fibers:
#     cur.execute("INSERT INTO fibers(name, material, modulus, uts, rho, comment) VALUES (?,?,?,?,?,?)", tuple(fiber))

# cur.close()
# conn.commit()

Create matrix table

In [37]:
tb_matricies = ["name TEXT",
                "material INT",
                "E REAL",
                "G REAL",
                "nu REAL",
                "rho REAL",
                "uts REAL",
                "firesafe BOOLEAN",
                "comment TEXT",
                "FOREIGN KEY(material) REFERENCES materials(rowid)"]

cur = conn.cursor()

cur.execute(createTableStr("matricies",tb_matricies))
cur.close()
conn.commit()

Insert dummy data into matricies table

In [38]:
list_matricies = [["DT120", 3, 100, 40, 0.3, 50, 20, True, "no comment"]]

cur = conn.cursor()
for matrix in list_matricies:
    cur.execute("INSERT INTO matricies(name, material, e, g, nu, rho, uts, firesafe, comment) VALUES (?,?,?,?,?,?,?,?,?)", tuple(matrix))
cur.close()
conn.commit()



Create Laminas table

In [28]:
tb_laminas = ["name TEXT",
              "fiber INT",
              "use INT",
              "E1 REAL",
              "E2 REAL",
              "G12 REAL",
              "nu REAL",
              "vff REAL",
              "faw REAL", 
              "thickness REAL",
              "matrix REAL",
              "weave INT",
              "comment TEXT",
              "FOREIGN KEY(fiber) REFERENCES fibers(rowid)",
              "FOREIGN KEY(use) REFERENCES usecases(rowid)",
              "FOREIGN KEY(matrix) REFERENCES matricies(rowid)",
              "FOREIGN KEY(weave) REFERENCES weaves(rowid)"]

cur = conn.cursor()
cur.execute(createTableStr("laminas", tb_laminas))
cur.close()
conn.commit()

In [27]:
cur = conn.cursor()

cur.execute("DROP TABLE laminas")
cur.close()
conn.commit()

Add dummy data to Laminas table

In [29]:
list_laminas = [["IMS60", 2, 4, 200, 200, 150, 0.3, 0.5, 120, 0.102, 1, 1, "test ply 1"],
                ["Akku aramid", 1, 6, 20, 20, 10, 0.3, 0.45, 150, 0.02, 1, 1, "test ply 2"]]


cur = conn.cursor()

for lamina in list_laminas:
    cur.execute("INSERT INTO laminas VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)", tuple(lamina))

cur.close()
conn.commit()

OperationalError: table laminas has 13 columns but 12 values were supplied

In [12]:
cur = conn.cursor()
data = cur.execute("select * from laminas").fetchall()
cur.close()

In [20]:
cur = conn.cursor()
cur.execute("SELECT rowid FROM materials WHERE name IS 'Carbon'").fetchall()[0][0]

2